In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os
root_depth = 3
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [30]:
import numpy as np
import scipy as sp
import pandas as pd
import functools
import holoviews as hv
from holoviews import dim, opts

# Spykshrk modules for data analysis
from spykshrk.franklab.data_containers import FlatLinearPosition, SpikeFeatures, \
        EncodeSettings, pos_col_format, SpikeObservation
from spykshrk.franklab.generator.place.data_generator import UnitNormalGenerator, TetrodeUniformUnitNormalGenerator
from spykshrk.franklab.pp_decoder.util import normal_pdf_int_lookup, gaussian
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPEncoder, OfflinePPDecoder
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.visualization import LinPosVisualizer, TetrodeVisualizer
from spykshrk.util import AttrDict

hv.extension('bokeh')
hv.extension('matplotlib')

hv.renderer('bokeh').theme = "dark_minimal"
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

In [3]:
# Encoding and decoding settings for both simulator and algorithm
encode_settings = AttrDict({'sampling_rate': 1000,
                            'pos_bins': np.arange(0,100,1),
                            'pos_bin_edges': np.arange(0,100.1,1),
                            'pos_bin_delta': 1,
                            'pos_kernel': sp.stats.norm.pdf(np.arange(0,100,1), 50, 1),
                            'pos_kernel_std': 1, 
                            'mark_kernel_std': int(20), 
                            'pos_num_bins': 100,
                            'pos_col_names': [pos_col_format(ii, 100) for ii in range(100)],
                            'arm_coordinates': [[0,100]],
                            'vel': 3,
                            'spk_amp': 60})

decode_settings = AttrDict({'trans_smooth_std': 5,
                            'trans_uniform_gain': 0.001,
                            'time_bin_size': 10})
                            

In [4]:
tet_gen = TetrodeUniformUnitNormalGenerator(sampling_rate=encode_settings.sampling_rate,
                                            num_marks=4,
                                            num_units=50,
                                            mark_mean_range=(60, 200),
                                            mark_cov_range=(20, 40),
                                            firing_rate_range=(5, 40),
                                            pos_field_range=(0, 100),
                                            pos_field_var_range=(2,10))

In [80]:

sampling_rate = 1000

# Simulate Synthetic Position Data
# Animal runs back and forth between two points, pausing for a few seconds in the middle.
pos_time = np.arange(0,100*1000,1)
pos_run = 50*np.cos(pos_time[0:17272]/(500*np.pi))+50
pos_run = np.append(pos_run, ([pos_run[-1]]*39478))
pos_run = np.append(pos_run, 50*np.cos(pos_time[56750:4000*1000]/(500*np.pi))+50)
pos_vel = np.concatenate([[0], np.diff(pos_run) * sampling_rate])

pos_vel = np.concatenate([[0], np.diff(pos_run) * encode_settings.sampling_rate])

linpos_flat_obj = FlatLinearPosition.from_numpy_single_epoch(1, 1, pos_time, pos_run, pos_vel, encode_settings.sampling_rate, 
                                                             encode_settings.arm_coordinates)


In [81]:
hv.output(backend='bokeh')
linpos_viz = LinPosVisualizer(linpos_flat_obj)
linpos_viz.plot_linpos_flat()

In [36]:
hv.Store.current_backend

In [6]:
spk_amps, unit_spks = tet_gen.simulate_tetrode_over_pos(linpos_flat_obj)

In [47]:
tet_viz = TetrodeVisualizer(spk_amps, linpos_flat_obj, unit_spks)
tet_viz.plot_color_3d_dynamic('linpos_flat', 'c00', 'c01')

In [8]:
tet_viz.plot_color_3d_dynamic('c00', 'c01', 'c02')